## Components in LlamaIndex

Alfred is hosting a party and needs to be able to find relevant information on personas that will be attending the party. Therefore, we will use a QueryEngine to index and search through a database of personas.

Let's install the dependencies.

We will install the dependencies for this unit.

```python
!pip install llama-index datasets llama-index-callbacks-arize-phoenix arize-phoenix llama-index-vector-stores-chroma llama-index-llms-huggingface-api llama-index-embeddings-huggingface -U -q
```

### Create a QueryEngine for retrieval augmented generation
##### Setting up the persona database
We will be using personas from the dvilasuero/finepersonas-v0.1-tiny dataset. This dataset contains 5K personas that will be attending the party!

Let's load the dataset and store it as files in the data directory

In [1]:
from datasets import load_dataset
from pathlib import Path

dataset = load_dataset(path='dvilasuero/finepersonas-v0.1-tiny', split="train")

Path("data").mkdir(parents=True, exist_ok=True)
for i, persona in enumerate(dataset):
    with open(Path("data") / f"persona_{i}.txt", "w", encoding="utf-8") as f:
        f.write(persona["persona"])

Awesome, now we have a local directory with all the personas that will be attending the party, we can load and index!

### Loading and embedding persona documents

We will use the SimpleDirectoryReader to load the persona descriptions from the data directory. This will return a list of Document objects.

In [2]:
from llama_index.core import SimpleDirectoryReader

reader = SimpleDirectoryReader(input_dir="data")
documents = reader.load_data()
len(documents)

5000

Now we have a list of Document objects, we can use the IngestionPipeline to create nodes from the documents and prepare them for the QueryEngine. We will use the SentenceSplitter to split the documents into smaller chunks and the HuggingFaceEmbedding to embed the chunks.

In [3]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.ingestion import IngestionPipeline

# create the pipeline with transformations
pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(),
        HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
    ]
)

# run the pipeline sync or async
nodes = await pipeline.arun(documents=documents[:10])
nodes

2026-01-10 00:08:41,550 - INFO - Load pretrained SentenceTransformer: BAAI/bge-small-en-v1.5
2026-01-10 00:08:45,867 - INFO - 1 prompt is loaded, with the key: query


[TextNode(id_='ff63473e-96bb-4477-aecf-9509f363d768', embedding=[-0.020338525995612144, 0.009064318612217903, 0.02818886935710907, -0.009143168106675148, -0.0014468986773863435, -0.03916342556476593, -0.0008997684344649315, 0.0022380324080586433, -0.045916441828012466, -0.07419972121715546, 0.01952705718576908, -0.017656011506915092, -0.007917121984064579, 0.02225489541888237, 0.0051278285682201385, 0.014362321235239506, -0.007187457289546728, 0.0496448315680027, 0.008878462016582489, -0.01440107636153698, -0.0046846517361700535, -0.048192672431468964, 0.05416930839419365, -0.05014868080615997, -0.05608641728758812, 0.03483348712325096, 0.04314301908016205, -0.03587701916694641, -0.009587574750185013, -0.15302398800849915, -0.025394316762685776, -0.003082113806158304, 0.028299888595938683, 0.023583898320794106, 0.06284604221582413, 0.014480403624475002, -0.0007606146391481161, 0.06458743661642075, -0.004938895348459482, -0.0034157384652644396, 0.0023614950478076935, 0.00841465778648853

As, you can see, we have created a list of `Node` objects, which are just chunks of text from original documents. let's explore how we can add these nodes to a vector store.

#### Storing and indexing documents

Since we are using an ingestion pipeline, we can directly attach a vector store to the pipeline to populate it. In this case, we will use `Chroma` to store our documents. Let's run the pipeline again with the vector store attached. The `IngestionPipeline` caches the operations so this should be fast.

In [4]:
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore

db = chromadb.PersistentClient(path="./afred_chroma.db")
chroma_collection = db.get_or_create_collection(name="alfred")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(),
        HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
    ],
    vector_store=vector_store
)

nodes = await pipeline.arun(documents=documents[:10])
len(nodes)

2026-01-10 00:08:47,596 - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2026-01-10 00:08:47,925 - INFO - Load pretrained SentenceTransformer: BAAI/bge-small-en-v1.5
2026-01-10 00:08:52,232 - INFO - 1 prompt is loaded, with the key: query


10

We can create a `VectorStoreIndex` from the vector store and use it to query the documents by passing the vector store and embedding model to the `from_vector_store()` method.

In [5]:
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store, embed_model=embed_model
)

2026-01-10 00:08:52,893 - INFO - Load pretrained SentenceTransformer: BAAI/bge-small-en-v1.5
2026-01-10 00:08:57,663 - INFO - 1 prompt is loaded, with the key: query


We don't need to worry about persisting the index to disk, as it is automatically saved within the `ChromaVectorStore` object and the passed directory path.

#### Querying the index

Now that we have our index, we can use it to query the documents. Let's create a `QueryEngine` from the index and use it to query the documents using a specific response node.

In [6]:
from llama_index.llms.ollama import Ollama
import nest_asyncio

nest_asyncio.apply()
llm = Ollama(model="kimi-k2:1t-cloud", base_url='http://127.0.0.1:11434')
query_engine = index.as_query_engine(
    llm=llm,
    response_model="tree_summarize",
)

response = query_engine.query(
    "Respond using a persona that describes author and travel experiences?"
)

response

2026-01-10 00:09:00,557 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/show "HTTP/1.1 200 OK"
2026-01-10 00:09:08,081 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


Response(response='I’ve spent years walking the stone-laced villages of the Troodos foothills, drinking thick, cardamom-scented coffee with elders who still measure time by the sound of church bells rather than clocks. My notebooks are crowded with sketches of lace-makers’ fingers dancing above spider-web fine thread in Lefkara, and with the salt-crusted stories of fishermen who launch their painted boats at dawn from the tiny coves near Ayia Napa, long before the DJs arrive. I’ve slept in Ottoman-era caravanserai turned family homes in the old quarters of Nicosia, woken to the call of the muezzin mingling with Sunday church bells, and traced the scent of wild thyme across the Akamas peninsula until it dissolved into turquoise water. Each journey here is less an itinerary than a conversation—one that begins with a handshake, a slice of halloumi warm from the grill, and the inevitable question: “Where are your grandparents from?”', source_nodes=[NodeWithScore(node=TextNode(id_='65da7b88

### Evaluation & observability

LlamaIndex provides <b>built-in evaluation tools to assess response quality.</b> These evaluators leverage LLMs to analyze responses across difference dimensions. We can now check if the query is faithful to the original persona.

In [7]:
from llama_index.core.evaluation import FaithfulnessEvaluator

# query index
evaluator = FaithfulnessEvaluator(llm=llm)
eval_result = evaluator.evaluate_response(response=response)
eval_result.passing

2026-01-10 00:09:10,774 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


True

If one of these LLM based evaluators does not give enough context, we can check the response using the Arize Phoenix tool, after creating an account at LlamaTrace and generating an API key.

In [10]:
import llama_index
import os
from dotenv import load_dotenv
load_dotenv()

PHOENIX_API_KEY = os.environ.get("PHOENIX_API_KEY")
os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"api_key={PHOENIX_API_KEY}"
llama_index.core.set_global_handler(
    "arize_phoenix", endpoint="https://llamatrace.com/v1/traces"
)

2026-01-10 00:12:44,752 - WARNING - Attempting to instrument while already instrumented


Now, we can query the index and see the response in the Arize Phoenix tool.

In [11]:
response = query_engine.query(
    "What is the name of the someone that is interested in AI and technology?"
)

response

2026-01-10 00:12:51,216 - ERROR - Failed to export span batch code: 401, reason: 
2026-01-10 00:12:51,492 - ERROR - Failed to export span batch code: 401, reason: 
2026-01-10 00:12:51,775 - ERROR - Failed to export span batch code: 401, reason: 
2026-01-10 00:12:52,136 - ERROR - Failed to export span batch code: 401, reason: 
2026-01-10 00:12:52,420 - ERROR - Failed to export span batch code: 401, reason: 
2026-01-10 00:12:52,697 - ERROR - Failed to export span batch code: 401, reason: 
2026-01-10 00:12:55,711 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2026-01-10 00:12:55,983 - ERROR - Failed to export span batch code: 401, reason: 
2026-01-10 00:12:56,256 - ERROR - Failed to export span batch code: 401, reason: 
2026-01-10 00:12:56,638 - ERROR - Failed to export span batch code: 401, reason: 
2026-01-10 00:12:56,909 - ERROR - Failed to export span batch code: 401, reason: 
2026-01-10 00:12:57,180 - ERROR - Failed to export span batch code: 401, reaso

Response(response='The provided information does not mention anyone interested in AI and technology.', source_nodes=[NodeWithScore(node=TextNode(id_='65da7b88-8760-4a14-81f4-9783f49eba3a', embedding=None, metadata={'file_path': "c:\\Users\\bhask\\OneDrive\\Documents\\Learning\\Huggingface AI Agents's Course\\llamaindex_framework\\data\\persona_1.txt", 'file_name': 'persona_1.txt', 'file_type': 'text/plain', 'file_size': 266, 'creation_date': '2026-01-09', 'last_modified_date': '2026-01-09'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='076047bf-5bcf-4277-b352-8e879659488c', node_type='4', metadata={'file_path': "c:\\Users\\bhask\\OneDrive\\Documents\\Learning\\Huggingface AI Agents's Course\\llamaindex_f